https://github.com/dtrckd/simplon_datai_2020/blob/master/brief_7/brief.md

https://www.dataquest.io/blog/last-fm-api-python/

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html#sklearn.model_selection.GridSearchCV

https://towardsdatascience.com/recommender-systems-and-hyper-parameter-tuning-25567b10e298

In [2]:
import pandas as pd
import numpy as np

In [3]:
plays = pd.read_csv('user_artists.dat', sep='\t')

plays.head(5)

,userID,artistID,weight
0,2,51,13883
1,2,52,11690
2,2,53,11351
3,2,54,10300
4,2,55,8983


In [4]:
artists = pd.read_csv('artists.dat', sep='\t')#, usecols=['id','name'])
artists.head(5)

,id,name,url,pictureURL
0,1,MALICE MIZER,http://www.last.fm/music/MALICE+MIZER,http://userserve-ak.last.fm/serve/252/10808.jpg
1,2,Diary of Dreams,http://www.last.fm/music/Diary+of+Dreams,http://userserve-ak.last.fm/serve/252/3052066.jpg
2,3,Carpathian Forest,http://www.last.fm/music/Carpathian+Forest,http://userserve-ak.last.fm/serve/252/40222717...
3,4,Moi dix Mois,http://www.last.fm/music/Moi+dix+Mois,http://userserve-ak.last.fm/serve/252/54697835...
4,5,Bella Morte,http://www.last.fm/music/Bella+Morte,http://userserve-ak.last.fm/serve/252/14789013...


In [5]:
artists = pd.read_csv('artists.dat', sep='\t', usecols=['id','name'])
artists.head(5)

,id,name
0,1,MALICE MIZER
1,2,Diary of Dreams
2,3,Carpathian Forest
3,4,Moi dix Mois
4,5,Bella Morte


In [6]:
# Merge artist and user pref data (Fusionner les données de préférence de l'artiste et de l'utilisateur)
ap = pd.merge(artists, plays, how="inner", left_on="id", right_on="artistID") 
ap.head(5)


,id,name,userID,artistID,weight
0,1,MALICE MIZER,34,1,212
1,1,MALICE MIZER,274,1,483
2,1,MALICE MIZER,785,1,76
3,2,Diary of Dreams,135,2,1021
4,2,Diary of Dreams,257,2,152


In [7]:
ap = ap.rename(columns={"weight": "playCount"})
ap.head(5)

,id,name,userID,artistID,playCount
0,1,MALICE MIZER,34,1,212
1,1,MALICE MIZER,274,1,483
2,1,MALICE MIZER,785,1,76
3,2,Diary of Dreams,135,2,1021
4,2,Diary of Dreams,257,2,152


In [8]:
# Group artist by name
artist_rank = ap.groupby(['name']) \
    .agg({'userID' : 'count', 'playCount' : 'sum'}) \
    .rename(columns={"userID" : 'totalUsers', "playCount" : "totalPlays"}) \
    .sort_values(['totalPlays'], ascending=False)

artist_rank['avgPlays'] = artist_rank['totalPlays'] / artist_rank['totalUsers']
print(artist_rank)


                    totalUsers  totalPlays     avgPlays
name                                                   
Britney Spears             522     2393140  4584.559387
Depeche Mode               282     1301308  4614.567376
Lady Gaga                  611     1291387  2113.563011
Christina Aguilera         407     1058405  2600.503686
Paramore                   399      963449  2414.659148
...                        ...         ...          ...
Morris                       1           1     1.000000
Eddie Kendricks              1           1     1.000000
Excess Pressure              1           1     1.000000
My Mine                      1           1     1.000000
A.M. Architect               1           1     1.000000

[17632 rows x 3 columns]


In [9]:
# Merge into ap matrix #join rajoute les colonnes de artist rank sur ap en correlant sur le nom des artiste (name)
ap = ap.join(artist_rank, on="name", how="inner") \
    .sort_values(['playCount'], ascending=False)
ap.head(10)

,id,name,userID,artistID,playCount,totalUsers,totalPlays,avgPlays
2800,72,Depeche Mode,1642,72,352698,282,1301308,4614.567376
35843,792,Thalía,2071,792,324663,26,350035,13462.884615
27302,511,U2,1094,511,320725,185,493024,2664.994595
8152,203,Blur,1905,203,257978,114,318221,2791.412281
26670,498,Paramore,1664,498,227829,399,963449,2414.659148
21191,378,Evanescence,1146,378,203165,226,513476,2272.017699
14165,292,Christina Aguilera,514,292,176133,407,1058405,2600.503686
33309,701,Shakira,1983,701,172496,319,688529,2158.398119
33127,701,Shakira,757,701,169596,319,688529,2158.398119
31835,679,Glee Cast,1086,679,165902,249,506453,2033.947791


In [10]:
ap.sort_values(['userID'], ascending=False)[:10]

,id,name,userID,artistID,playCount,totalUsers,totalPlays,avgPlays
234,12,Behemoth,2100,12,1253,19,9564,503.368421
17,3,Carpathian Forest,2100,3,408,3,775,258.333333
61173,2749,Deathspell Omega,2100,2749,532,6,2943,490.500000
31,6,Moonspell,2100,6,404,10,5080,508.000000
75464,6258,Summoning,2100,6258,403,6,1842,307.000000
44041,1109,Yann Tiersen,2100,1109,1333,69,43972,637.275362
79923,8332,Luctus,2100,8332,655,3,824,274.666667
92820,18729,Atalyja,2100,18729,280,1,280,280.000000
92819,18728,Domantas Razauskas,2100,18728,281,1,281,281.000000
67397,3806,Rotting Christ,2100,3806,389,8,1913,239.125000


In [11]:
ap.sort_values(['artistID'], ascending=False)[:10]

,id,name,userID,artistID,playCount,totalUsers,totalPlays,avgPlays
92833,18745,Grzegorz Tomczak,585,18745,426,1,426,426.0
92832,18744,Oz Alchemist,454,18744,286,1,286,286.0
92831,18743,Coptic Rain,454,18743,287,1,287,287.0
92830,18742,Aya RL,454,18742,294,1,294,294.0
92829,18741,Diamanda Galás,454,18741,301,1,301,301.0
92828,18740,Die Krupps,454,18740,320,1,320,320.0
92827,18739,Apollo 440,454,18739,379,1,379,379.0
92826,18738,Cubanate,454,18738,408,1,408,408.0
92825,18737,Ciccone Youth,454,18737,560,1,560,560.0
92824,18736,LOSTFREEQ,454,18736,879,1,879,879.0


In [12]:
# Preprocessing
pc = ap.playCount
play_count_scaled = (pc - pc.min()) / (pc.max() - pc.min())
ap = ap.assign(playCountScaled=play_count_scaled)
ap.head(10)
#print(ap)


,id,name,userID,artistID,playCount,totalUsers,totalPlays,avgPlays,playCountScaled
2800,72,Depeche Mode,1642,72,352698,282,1301308,4614.567376,1.000000
35843,792,Thalía,2071,792,324663,26,350035,13462.884615,0.920513
27302,511,U2,1094,511,320725,185,493024,2664.994595,0.909347
8152,203,Blur,1905,203,257978,114,318221,2791.412281,0.731441
26670,498,Paramore,1664,498,227829,399,963449,2414.659148,0.645960
21191,378,Evanescence,1146,378,203165,226,513476,2272.017699,0.576030
14165,292,Christina Aguilera,514,292,176133,407,1058405,2600.503686,0.499386
33309,701,Shakira,1983,701,172496,319,688529,2158.398119,0.489074
33127,701,Shakira,757,701,169596,319,688529,2158.398119,0.480852
31835,679,Glee Cast,1086,679,165902,249,506453,2033.947791,0.470378


In [13]:
# Build a user-artist rating matrix 
ratings_df = ap.pivot(index='userID', columns='artistID', values='playCountScaled')
ratings_df.head(5)
#.pivot Renvoie un DataFrame remodelé organisé par des valeurs d'index / colonne données. 

artistID,1,2,3,4,5,6,7,8,9,10,...,18736,18737,18738,18739,18740,18741,18742,18743,18744,18745
userID,,,,,,,,,,,,,,,,,,,,,
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
ratings = ratings_df.fillna(0).values
ratings_df.head(5)
#Remplissez les valeurs NA / NaN à l'aide de la méthode spécifiée.

artistID,1,2,3,4,5,6,7,8,9,10,...,18736,18737,18738,18739,18740,18741,18742,18743,18744,18745
userID,,,,,,,,,,,,,,,,,,,,,
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
ratings

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.00115397, ..., 0.        , 0.        ,
        0.        ]])

In [16]:
# Show sparsity  #.nonzero Renvoie les indices des éléments non nuls. 
sparsity = float(len(ratings.nonzero()[0])) / (ratings.shape[0] * ratings.shape[1]) * 100
print("sparsity: %.2f" % sparsity)

sparsity: 0.28


In [17]:
from scipy.sparse import csr_matrix

# Build a sparse matrix
X = csr_matrix(ratings)

n_users, n_items = ratings_df.shape
print("rating matrix shape", ratings_df.shape)

user_ids = ratings_df.index.values
artist_names = ap.sort_values("artistID")["name"].unique()

rating matrix shape (1892, 17632)


In [18]:
artist_names

array(['MALICE MIZER', 'Diary of Dreams', 'Carpathian Forest', ...,
       'Coptic Rain', 'Oz Alchemist', 'Grzegorz Tomczak'], dtype=object)

In [19]:
ratings_df.shape


(1892, 17632)

In [20]:
n_users

1892

In [21]:
user_ids

array([   2,    3,    4, ..., 2097, 2099, 2100])

In [22]:
X

<1892x17632 sparse matrix of type '<class 'numpy.float64'>'
	with 92198 stored elements in Compressed Sparse Row format>

In [23]:
from lightfm import LightFM
from lightfm.evaluation import auc_score, precision_at_k, recall_at_k
from lightfm.cross_validation import random_train_test_split
from lightfm.data import Dataset

# Build data references + train test
Xcoo = X.tocoo() #Renvoie une représentation COOrdinate de cette matrice 
data = Dataset()
data.fit(np.arange(n_users), np.arange(n_items)) #Renvoie des valeurs régulièrement espacées dans un intervalle donné. 
interactions, weights = data.build_interactions(zip(Xcoo.row, Xcoo.col, Xcoo.data)) 
#Appelez build_interactions avec un itérable de (user id, item id) ou (user id, item id, weight
#pour construire une matrice d'interactions et de poids. 
train, test = random_train_test_split(interactions,random_state=42)

# Ignore that (weight seems to be ignored...)
#train = train_.tocsr()
#test = test_.tocsr()
#train[train==1] = X[train==1]
#test[test==1] = X[test==1]

# To be completed...

In [24]:
# Train
model = LightFM(learning_rate=0.05, loss='warp',random_state=42)
model.fit(train, epochs=10, num_threads=2)

In [25]:
# Evaluate
train_precision = precision_at_k(model, train, k=10).mean()
test_precision = precision_at_k(model, test, k=10, train_interactions=train).mean()

train_auc = auc_score(model, train).mean()
test_auc = auc_score(model, test, train_interactions=train).mean()

print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))
print('AUC: train %.2f, test %.2f.' % (train_auc, test_auc))

Precision: train 0.38, test 0.13.
AUC: train 0.97, test 0.86.


In [26]:
# Optimisation de paramètres avec GridSearch

from sklearn.model_selection import ParameterGrid# Create the parameter grid based on the results of random search 

#creation du dictionnaire avec les parametres a tester, combiner
param_grid = {
    'learning_rate': [0.05 , 0.08],
    'learning_schedule':['adagrad','adadelta'],
    'loss': ['warp','bpr','logistic','warp-kos']

}

list(ParameterGrid(param_grid)) #renvoi la liste des combinaison possible

[{'learning_rate': 0.05, 'learning_schedule': 'adagrad', 'loss': 'warp'},
 {'learning_rate': 0.05, 'learning_schedule': 'adagrad', 'loss': 'bpr'},
 {'learning_rate': 0.05, 'learning_schedule': 'adagrad', 'loss': 'logistic'},
 {'learning_rate': 0.05, 'learning_schedule': 'adagrad', 'loss': 'warp-kos'},
 {'learning_rate': 0.05, 'learning_schedule': 'adadelta', 'loss': 'warp'},
 {'learning_rate': 0.05, 'learning_schedule': 'adadelta', 'loss': 'bpr'},
 {'learning_rate': 0.05, 'learning_schedule': 'adadelta', 'loss': 'logistic'},
 {'learning_rate': 0.05, 'learning_schedule': 'adadelta', 'loss': 'warp-kos'},
 {'learning_rate': 0.08, 'learning_schedule': 'adagrad', 'loss': 'warp'},
 {'learning_rate': 0.08, 'learning_schedule': 'adagrad', 'loss': 'bpr'},
 {'learning_rate': 0.08, 'learning_schedule': 'adagrad', 'loss': 'logistic'},
 {'learning_rate': 0.08, 'learning_schedule': 'adagrad', 'loss': 'warp-kos'},
 {'learning_rate': 0.08, 'learning_schedule': 'adadelta', 'loss': 'warp'},
 {'learning_

In [27]:
auc_score_values = [] #declare une liste

for grid in ParameterGrid(param_grid):
    model = LightFM(**grid) #double etoile applique grid au model
    pred = model.fit(train) #fit applique le model au train
    auc_score_values.append(round(auc_score(model, test, train_interactions=train).mean(),3))
    #append ajoute le resulta de la boucle dans la liste declaré au fur et a mesur. ARRONDI ET MOYENNE (le,3 dit arrondi 3 chiffre apres la virgule)
    # on lui met la valeur de l'auc

max_value = max(auc_score_values) #valeur max de la liste auc score values
max_index = auc_score_values.index(max_value) # recup lindex de cette valeur max ou #np.argmax(auc_scores_values)
ParameterGrid(param_grid)[max_index ].items() 
# on rappel la liste de toute les combinaison, on applique la fonction max index et le .items renvoi la couple cles et valeur du dictionnaire param_grid


dict_items([('loss', 'warp'), ('learning_schedule', 'adadelta'), ('learning_rate', 0.08)])

In [28]:
# Predict
scores = model.predict(0, np.arange(n_items))
top_items = artist_names[np.argsort(-scores)]
print(top_items)

['The Cure' 'The Clash' 'Radiohead' ... 'The Passage' 'Low' 'Pryda']


In [29]:
def get_recommendations(userID):
    scores = model.predict(userID, np.arange(n_items))
    top_items = artist_names[np.argsort(-scores)]
    return top_items

In [30]:
get_recommendations(0)

array(['The Cure', 'The Clash', 'Radiohead', ..., 'The Passage', 'Low',
       'Pryda'], dtype=object)

In [31]:
#get_ground_truth qui renvoie les artistes ecoutés par un utilisateur par ordre décroissant du playCountScaled 

def get_ground_truth(userID):
    scores = model.predict(userID, np.arange(n_items))
    top_items = artist_names[np.argsort(-scores)]
    
    
    ap.sort_values(['playCountScaled'], ascending=False)
    return artiste ecouté


#relier user id avec artist name et playcountscaled

SyntaxError: invalid syntax (<ipython-input-31-3c383e90600e>, line 9)

In [ ]:
def get_ground_truth_artists_for_user(userID):
    art = list(enumerate(ratings[userID]))
    art = sorted(art, key=lambda x: x[1], reverse=True)
    art = art[1:10]
    art_indx = [i[0] for i in art]
    artistes = []
    for i in art_indx:
        artistes.append(ap['name'][ap['artistID']==i].unique()[0])
    return artistes

In [ ]:
get_ground_truth_artists_for_user(0)

https://www.digitalocean.com/community/tutorials/how-to-make-a-web-application-using-flask-in-python-3-fr

https://mensuel.framapad.org/p/vl6g3xporc-9lgd?lang=en

https://github.com/inboxpraveen/recommendation-system/blob/master/templates/index.html

http://www.xavierdupre.fr/app/ensae_teaching_cs/helpsphinx3/notebooks/TD2A_eco_debuter_flask.html#le-template-home-html

https://medium.com/analytics-vidhya/movie-recommendation-system-python-flask-web-application-heroku-deployment-7e39492b640c

In [ ]:
tuple(ap['name'].unique())